In [1]:
import polars as pl
import os
from tqdm import tqdm
from IPython.display import Markdown
import kagglehub
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from openai.types.chat.chat_completion_message_param import ChatCompletionMessageParam

sys.path.append("..")
from synthetic_data.screenplay_parser import ScreenplayParser

%load_ext autoreload
%autoreload 2

/home/brianf/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
scripts_corpus_path = kagglehub.dataset_download("veeralakrishna/imsdb-movie-scripts")
print(scripts_corpus_path)
scripts_pqt_path = os.path.join(scripts_corpus_path, "movie_scripts.parquet")
scripts_pqt = pl.read_parquet(scripts_pqt_path)

100%|██████████| 93.5M/93.5M [00:04<00:00, 19.8MB/s]

Extracting files...


/home/brianf/.cache/kagglehub/datasets/veeralakrishna/imsdb-movie-scripts/versions/2


In [ ]:
for i in range(10):
    display(Markdown(f"## {scripts_pqt[i]['Movie'][0]}"))
    print(scripts_pqt[i]["Script"][0][0:1000])

In [ ]:
scene_counts, char_line_counts = [], []
i, n_failed = 0, 0
good_scripts = []
for row in tqdm(scripts_pqt.iter_rows(named=True)):
    i += 1
    parser = ScreenplayParser(row["Script"])
    scenes = parser.parse()
    n_chars, n_chars_with_lines = len(scenes), len(parser.character_line_counts)
    scene_counts.append(n_chars)
    char_line_counts.append(n_chars_with_lines)
    if n_chars_with_lines >= 10 and n_chars >= 30:
        good_scripts.append(parser)
    if n_chars_with_lines == 0 and n_chars == 0:
        # print(row['Script'])
        n_failed += 1
        continue
    # print(f"Script {i} has {n_chars} scenes and {n_chars_with_lines} characters with lines")
    # for scene in scenes:
    #     display(Markdown(f"### {scene.heading.content}"))
    #     for elem in scene.elements:
    #         if elem.type == 'dialogue':
    #             if not elem.dialogue_data:
    #                 raise ValueError("Dialogue element has no dialogue data")
    #             display(Markdown(f"**{elem.dialogue_data.character}**: {elem.dialogue_data.content}"))

print(f"Failed to parse {n_failed} scripts, got {len(good_scripts)} good scripts")

In [ ]:
scene_counts_np = pl.Series("char_counts", scene_counts).to_numpy()
char_line_counts_np = pl.Series("char_line_counts", char_line_counts).to_numpy()

pcts = [40, 50, 60]
print(pcts)
scene_ct_pcts = np.percentile(scene_counts_np, pcts)
char_ct_pcts = np.percentile(char_line_counts_np, pcts)

print(f"char_cpts: {char_ct_pcts} scene_ct_percentiles: {scene_ct_pcts}")

# Plotting KDE with percentiles
plt.figure(figsize=(12, 8))

# KDE plots
sns.kdeplot(scene_counts_np, label="Scene Counts", fill=True, color="blue", alpha=0.5)

# Mark percentiles on the plot
for i, p in enumerate(scene_ct_pcts):
    plt.axvline(
        p,
        color="blue",
        linestyle="--",
        alpha=0.8,
        label=f"Scene Count Percentile {i}: {p:.1f}",
    )
plt.legend()

plt.show()
sns.kdeplot(
    char_line_counts_np, label="Char Line Counts", fill=True, color="orange", alpha=0.5
)

for i, p in enumerate(char_ct_pcts):
    plt.axvline(
        p,
        color="orange",
        linestyle="--",
        alpha=0.8,
        label=f"Char Line Count Percentile {i}: {p:.1f}",
    )

plt.legend()

plt.show()

In [2]:
# Conversation formatting

summarized_scenes = pl.read_parquet("../screenplay_scenes_summarized_full.parquet")
formatted_samples = []
for sample in tqdm(summarized_scenes.iter_rows(named=True)):
    scene: str = sample["scene"]
    summary: str = sample["summary"]
    name: str = sample["name"]
    if len(scene.split("\n")) < 10:
        continue
    conversation: list[ChatCompletionMessageParam] = [
        {
            "role": "system",
            "content": "You are an expert AI screenwriter. Your task is to take a user provided summary of a scene and generate a screenplay scene based on it.",
        },
        {"role": "user", "content": f"<summary>{summary}</summary>"},
        {"role": "assistant", "content": f"<scene>{scene}</scene>"},
    ]
    formatted_samples.append({"conversation": conversation, "name": name})

formatted_samples_pqt = pl.DataFrame(formatted_samples)
formatted_samples_pqt.write_parquet("screenplay_scenes_chat_fomrat.parquet")

FileNotFoundError: No such file or directory (os error 2): ../screenplay_scenes_summarized_full.parquet